### Lab: KFold, Regularization, & Pipelines

Welcome!  This lab is going to introduce us to some very important aspects of data processing and model building.  

Specifically, it's going to go over the following:

 - **KFold Cross Validation:** This is a more thorough way of choosing your validation set to give you a better idea of how your model might perform under various circumstances within your training data.
 - **Regularization:** This is an evergreen technique for dealing with models that are overfit (ie, higher scores on training vs. test data).  Regularized linear models are often much better prepared to handle messy data & outliers when using this technique.
 - **Pipelines:** (Time permitting!) This is an underappreciated aspect of the Scikit-Learn api that allows you to chain together multiple data processing steps, making it much easier to test different models and work seamlessly between your training & test sets.

**Note:** This lab builds off of the one performed in the last class.  As such, it might be easier just to keep working in your previous lab to answer these questions.  It assumes you already have your data processed from the iowa housing lab.  

The questions are listed here just to make the separation of concerns easier.

### Question 1: How Does Your Validation Score Differ Using KFold Cross Validation?

Take a look at the validation score you got from your previous exercise.  

This time, run your model through KFold cross validation using `cross_val_score`.  Is your total validation score appreciably different?  What were your highest and lowest values?

What if you changed your number of folds?  Try using 5, 10, & 25 folds.

In [7]:
# these are the steps from the previous lab
import pandas as pd
import numpy as np
train = pd.read_csv('../data/iowa_housing/train.csv')
test  = pd.read_csv('../data/iowa_housing/test.csv')

# your answer here
y = train['SalePrice']
train.drop('SalePrice', axis=1, inplace=True)
train.drop('Id', axis=1, inplace=True)
test.drop('Id', axis=1, inplace=True)

train_empty = train.loc[:, train.isnull().sum() > 0]
# grab the columns
cols = train_empty.columns.tolist()
# fill with the appropriate value  -- NA, Other, could also work
train[['GarageType', 'GarageFinish']] = train[['GarageType', 'GarageFinish']].fillna('None')
test[['GarageType', 'GarageFinish']]  = test[['GarageType', 'GarageFinish']].fillna('None')

# we'll use this for GarageYrBlt since it's a numeric column
train['GarageYrBlt'].fillna(0, inplace=True)
test['GarageYrBlt'].fillna(0, inplace=True)

# finding the values to use in the training set
ms_mode   = train['MSZoning'].mode()[0]
gcarsmean = train['GarageCars'].mean()

# and applying them to the test set
test['MSZoning'].fillna(ms_mode, inplace=True)
test['GarageCars'].fillna(gcarsmean, inplace=True)

# your code here
# we'll assume the GarageFinish is ordinal.  Ie, FinishedGarage > Unfinished Garage
garage_mapping = {
    'None': 0, # no garage
    'Unf' : 1, # unfinished garage
    'RFn' : 2, # partially finished garage
    'Fin' : 3  # finished garage
}

train['GarageFinish'] = train['GarageFinish'].map(garage_mapping)
test['GarageFinish']  = test['GarageFinish'].map(garage_mapping)

# MSSubClass is really a category, moreso than a true number
# so we'll add it to the list of items to be encoded
train['MSSubClass'] = train['MSSubClass'].astype(str)
test['MSSubClass']  = test['MSSubClass'].astype(str)

# concatenate and encode
master = pd.concat([train, test])
master = pd.get_dummies(master)

# drop MSSubClass150
master.drop('MSSubClass_150', axis=1, inplace=True)

# and split back apart
train  = master.iloc[:1460].copy()
test   = master.iloc[1460:].copy()

# save these values, to use on both your training and test set
train_means = train.mean()
train_stds  = train.std()

# standardize the training set
train_std = train - train_means
train_std /= train_stds

# and do the same for the test set
test -= train_means
test /= train_stds

In [28]:
# this is the part where we use KFold to find your validation score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
lreg = LinearRegression()
# we'll use a loop to go through these
cv_scores = []
num_folds = [5, 10, 25]

for fold in num_folds:
    scores = cross_val_score(estimator=lreg, X=train_std, y=y, cv=fold)
    cv_scores.append(scores)

In [37]:
# first we'll look at the different scores
cv_dict = {}
for idx, fold in enumerate(num_folds):
    cv_dict[f'folds: {fold}'] = np.mean(cv_scores[idx])

In [39]:
# scores are roughly the same, but lower than just the general validation score
# we had before
cv_dict

{'folds: 5': 0.8157003069679292,
 'folds: 10': 0.8223537273852971,
 'folds: 25': 0.8120534743018664}

### Question 2: Updating Your Model With Ridge & Lasso Regression

Instead of using Linear Regression, import `Ridge` or `Lasso`, and use cross validation to find the ideal value of alpha.  

Some basic tips:

For values of alpha try this:  `alphas = np.logspace(-4, 4, 9)`
Then write a `for-loop` that generically goes like this:

`for value in alphas:
    1). set value of alpha to current value using set_params() method
    2). pass in instance of Ridge or Lasso into cross_val_score
    3). using a tuple, append the average of all results from step 2 into a list, along with the value of alpha`
    
When you're finished, you should have a list that has 9 tuples inside it, each one with the average cross validation score as well as the value of alpha associated with it.

In [41]:
# your answer here
from sklearn.linear_model import Ridge, Lasso
ridge = Ridge()
lasso = Lasso()
alphas = np.logspace(-4, 4, 9)
ridge_scores = []
lasso_scores = []

for value in alphas:
    ridge.set_params(alpha=value)
    lasso.set_params(alpha=value)
    lasso_score = cross_val_score(estimator=lasso, X=train_std, y=y, cv=10)
    ridge_score = cross_val_score(estimator=ridge, X=train_std, y=y, cv=10)
    ridge_scores.append((np.mean(ridge_score), value))
    lasso_scores.append((np.mean(lasso_score), value))

C:\Users\Jonat\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 699289009019.7437, tolerance: 852398977.1246126
  positive)
C:\Users\Jonat\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 686855228053.8666, tolerance: 827439703.8697118
  positive)
C:\Users\Jonat\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 698601655799.0824, tolerance: 829042909.2707292
  positive)
C:\Users\Jonat\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 

C:\Users\Jonat\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 678684491275.5731, tolerance: 829042909.2707292
  positive)
C:\Users\Jonat\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 614102917569.1448, tolerance: 820538852.0864891
  positive)
C:\Users\Jonat\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 557443880125.1488, tolerance: 785203388.8728524
  positive)
C:\Users\Jonat\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 

In [44]:
# the value of alpha that gives the best score is 100
max(ridge_scores)

(0.8265411636864535, 100.0)

**Bonus:** In Scikit-Learn cross validation is sometimes built into algorithms automatically.  Luckily this is the case with `Ridge` and `Lasso`.  If you're inclined to take a look at the `RidgeCV` and `LassoCV` methods, you can basically combine what we just did into one step.

**RidgeCV:** https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html
**LassoCV:** https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html

### Step 3: Building A Pipeline

Let's try building some pipelines to test out different versions of our models more easily.  

For this part......start with the version of your training and test set that already has its missing values filled in, but nothing more!

Create the following pipeline:

We'll skip ordinal encoding for now.

 - one of either Ridge or Lasso, StandardScaler, and the CategoryEncoders categorical encoder
 - make sure to use the `cols` argument with the categorical encoder to include numeric variables that are actually categories if this is necessary
 - use the best value of alpha from the previous step for either one
 - fit your model on the entire training set (because we're using the cross-validated best version of alpha)

In [72]:
# your answer here
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# initialize everything
ohe   = OneHotEncoder()
sc    = StandardScaler()
ridge = Ridge(alpha=100)
pipe  = make_pipeline(ohe, sc, ridge)

# and then fit it on the training set
pipe.fit(train, y)

Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=[], drop_invariant=False,
                               handle_missing='value', handle_unknown='value',
                               return_df=True, use_cat_names=False,
                               verbose=0)),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('ridge',
                 Ridge(alpha=100, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)